In [4]:
from PyPDF2 import PdfReader
def get_pdf_leave_text(pdf_docs):
    leave_text=""
    for pdf in pdf_docs:
        pdf_reader=PdfReader(pdf)
        
        for page_num in range(9,11):
            page = pdf_reader.pages[page_num]
            leave_text+=page.extract_text()
    return leave_text

In [5]:
leave_text = get_pdf_leave_text(["manager handbook policy in legato.pdf"])

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 250,
    chunk_overlap  = 50,
    length_function = len,
    add_start_index = True,
)
texts = text_splitter.create_documents([leave_text])

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
def get_vector_storedb(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
#     vector_store = FAISS.from_texts(texts=text_chunks,embedding=embeddings)
    vector_store = FAISS.from_documents(documents=text_chunks,embedding=embeddings)
    return vector_store
vector_store = get_vector_storedb(texts)

C:\ProgramData\Anaconda3\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer on your own. 
    Use three sentences maximum and keep the answer as concise as possible. 

    Follow below examples to give response:
    
    Question : Whom can a manger escalate to if there are issues with leaves ?
    ChatBot Answer: The manager should reach out to the HRBP aligned to their tower.    
    Question : An associate has exhausted all the emergency leaves and  unable to recover from COVID?
    ChatBot Answer: The maximum emergency leaves that can be availed by any associate is 14 days. If you have exhausted emergency leaves then the associate has to avail leaves from their personal leave balance and take casual leave, sick leave, and annual leaves.    
    Question : An associate is suffering from one of the Critical Illnesses and 30 days are not enough for an associate to recover. What are the next steps?
    ChatBot Answer: The associate is supposed to exhaust his annual, casual, and sick leave. After all the leaves are exhausted, each case needs to be discussed separately qualifying for LWP.
    
    {context}
    Question: {question}
    ChatBot Answer: """
# Prompt
QA_CHAIN_PROMPT = PromptTemplate(
input_variables=["context", "question"],
template=template,
    )
from langchain.llms import GPT4All

GPT4All_llm_nous = GPT4All(
    model="./models/nous-hermes-13b.ggmlv3.q4_0.bin",
    max_tokens=2048,
)

Found model file at  ./models/nous-hermes-13b.ggmlv3.q4_0.bin


In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.llms import GPT4All



def get_response(question):
        
    similar_docs = vector_store.similarity_search(question)    
    print(similar_docs)
    # Chain
    GPT4All_llm_nous_chain = load_qa_chain(GPT4All_llm_nous, chain_type="stuff", prompt=QA_CHAIN_PROMPT)

    # Run
    result = GPT4All_llm_nous_chain({"input_documents": similar_docs, "question": question}, return_only_outputs=True)

    return result['output_text']

In [10]:
get_response("An associate has exhausted all the emergency leaves and  unable to recover from COVID?")

[Document(page_content='Emergency Leaves10 Days COVID cases , Laptop Replacement and New Joiners not \nreceiving the LaptopSubmission of medical documents of family member mandatory for availing emergency leave under this category.', metadata={'start_index': 4424}), Document(page_content='Emergency Leaves10 Days COVID cases , Laptop Replacement and New Joiners not receiving the Laptop\nComp Off (CO) Depending', metadata={'start_index': 3245}), Document(page_content="in case the associate's access \nhas not been resumed after an associate's BGV has turned Green from RedHRBP to decide and communicate the business on the Garden Leave and get it applied from back end by sending inputs to the HROPs Team", metadata={'start_index': 8345}), Document(page_content='aIn case an associate is unable to meet the complete target, the Reporting Authority can apply half day leave on associate’s behalf , if the associate has not been available for more than 4 hours.\n•', metadata={'start_index': 5058})]

' The maximum emergency leaves that can be availed by any associate is 14 days. If you have exhausted emergency leaves then the associate has to avail leaves from their personal leave balance and take casual leave, sick leave, and annual leaves.'

In [11]:
get_response("How many days can a associate take voluntary time off and what are the guidelines?")

[Document(page_content='Voluntary Time off 2 Days Voluntary time off will help associates to volunteer for NGOs supported by SWARA, our CSR program. - 2 days can be utilized at one go or minimum of two hours or more across multiple days.', metadata={'start_index': 996}), Document(page_content='Eligible associates receive up to 2 days of paid time in a year to volunteer for qualified \ncharities as part of our CSR program.', metadata={'start_index': 1210}), Document(page_content='off is earned . \nLeave Without Pay 90 Days This leave shall be provided up to maximum of 3 months (90 days) subject to exigencies \nof work, and based on prior approval by the manager/HR. \nLeave Category Days Guidelines Comments', metadata={'start_index': 3545}), Document(page_content='Comp Off (CO) Depending\nAssociates who are required to work on non-working days (public holiday or weekends) or have worked by earning a total OT of 9 hours.CO is valid\xa0 for 3 months from the day comp \noff is earned .', me

'2 days'